In [ ]:
! python --version

In [ ]:
! pip install diffusers transformers scipy

In [ ]:
! pip install accelerate

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

In [ ]:
model_id = "darkstorm2150/Protogen_v2.2_Official_Release"

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

In [ ]:
pipe.config

In [ ]:
pipe = pipe.to("cuda")

In [ ]:
! pip install pyTelegramBotAPI

In [ ]:
import telebot
from telebot import types

import re
import time
import os

In [ ]:
def generate(user_id, user_prompt):
    
    image = pipe(
        prompt = user_prompt,
        height = 512,
        width = 512,
        num_inference_steps = 100,
        guidance_scale = 3.5,
        num_images_per_prompt = 1
    ).images[0]
    
    user_image_path = f"./images/{user_id}.png"
    
    if os.path.exists(user_image_path):
        os.remove(user_image_path)
    
    try:
        image.save(user_image_path)
        return user_image_path
    except:
        bot.send_message(user_id, "Произошла ошибка, попробуйте позже.", parse_mode='html')
        return "./images/error.png"

In [ ]:
bot = telebot.TeleBot('6045436042:AAEa8N0i69hZIIuwH_vVGgcz429peGCbRZ0')

@bot.message_handler(content_types=["text"])
def mail(message):
    user_id = message.chat.id
    user_prompt = re.findall('/imagine (.*?)', message.text)
    
    if len(prompt) > 0:
        bot.send_message(user_id, "Подождите несколько секунд.", parse_mode='html')
        path = generate(user_id, user_prompt[0])
                
        with open(path, 'rb') as photo:
            bot.send_photo(user_id, photo)
    else:
        bot.send_message(user_id, "Введите запрос через команду <i>/imagine</i>", parse_mode='html')

In [ ]:
while True:
    try:
        bot.polling(non_stop=True, interval=0)
    except Exception as e:
        print(e)
        time.sleep(5)
        continue